## 3 - Faster Sentiment Analysis

>Ở Notebook trước (Notebook số 2) chúng ta đã áp dụng các phương pháp, kỹ thuật phổ biến trong bài toán phân tích cảm xúc và đạt được accuracy xấp xỉ 84%. Trong Notebook lần này, chúng ta sẽ xây dựng một model cho kết quả tương đương nhưng tốc độ huấn luyện nhanh hơn đáng kể và chỉ sử dụng một nửa parameters so với model ở Notebook số 2. Chúng ta sẽ xây dựng mô hình "FastText" dựa trên paper [Bag of Tricks for Efficient Text Classification](https://arxiv.org/pdf/1607.01759.pdf)


### Preparing Data

>Một trong những ý tưởng chính của paper "FastText" là: họ tính n-grams của input sentence và chèn chúng vào cuối câu. Trong bài này, chúng tôi sẽ sử dụng bi-grams (bi-grams là cặp tokens/word ĐỨNG GẦN NHAU TRONG CÂU).

> Ví dụ: <br>
>> Trong câu: "How are you ?", các bi-grams lần lượt là: "how are", "are you" và "you ?".

> Hàm `generate_bigrams` có đầu vào là câu đã được tokenized, sau đó tính bi-grams của câu, chèn vào cuối câu và trả về câu sau khi được xử lý.

In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [2]:
generate_bigrams(['this', 'is', 'a', 'test'])

['this', 'is', 'a', 'test', 'this is', 'a test', 'is a']

> TorchText `Field` có đối số `preprocessing`, khi ta truyền một hàm vào đối số này, hàm đó sẽ được áp dụng cho câu sau khi được tokenized nhưng trước khi được số hóa (số hóa = chuyển từ list các tokens thành list các indexs). Ta sẽ truyền hàm `generate_bigrams` và đối số `preprocessing`.

>Ở đây, chúng ta không sử dụng RNN do đó không thể áp dụng packed padded sequences, do đó không cần thiết lập tham số `include_lengths=True`.

In [3]:
import torch
import torchtext
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(  tokenize = 'spacy',\
                    tokenizer_language='en_core_web_sm',\
                    preprocessing = generate_bigrams)

LABEL = data.LabelField(dtype = torch.float)

In [4]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

> Xây dựng vocab và load pre-trained word embedding.

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

> Tạo iterators.

In [6]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = \
    data.BucketIterator.splits(
                                (train_data, valid_data, test_data),            
                                batch_size = BATCH_SIZE,
                                device = device)


### Build the Model

> Model này có khá ít tham số so với model ở Notebook 2, lý do là model chỉ có 2 layers chứa tham số: embedding layer và linear layer. Model không có thành phần RNN. 

> Đầu tiên, model sẽ tính word embedding cho mỗi từ với `Embedding` layer (blue), sau đó tính trung bình của tất cả word embeddings (pink), cuối cùng cho qua một lớp `Linear` layer (silver).

![LSTM](./images/3_model_architecture.PNG "LSTM")

> Tiếp theo, chúng tôi triển khai hàm `avg_pool2d` (average pool 2-dimensions). Tuy nhiên, 1 câu có chiều là 1, không phải 2 chiều tại sao lại dùng `avg_pool2d`? Câu trả lời là: khi ta embed câu, word embedding sẽ có số chiều là 2, mỗi hàng là 1 word embedding của 1 từ trong câu.

> Hình dưới minh họa word embedding với số chiều là 5, mỗi hàng là 1 embedding của từ, câu có 4 từ => kích thước của ma trận embedding là [4x5]:

![LSTM](./images/4_5_embedding_figure1.PNG "LSTM")


> Hàm `avg_pool2d` sử dụng một bộ lọc có kích thước `embedded.shape[1]` (ví dụ: chiều dài của câu). Bộ lọc được biểu diễn bởi màu hồng trong ảnh dưới:

![LSTM](./images/4_5_embedding_figure2.PNG "LSTM")


> Tính giá trị trung bình của các cột được cover bởi bộ lọc, sau đó bộ lọc trượt dần sang phải, tính giá trị trung bình của cột tiếp theo trong các giá trị embedding đối với mỗi từ trong câu.

![LSTM](./images/4_5_embedding_figure3.PNG "LSTM")


> Sau khi bộ lọc trượt hết trong không gian embedding, ta thu được một tensor có kích thước [1x5], tensor này được đưa vào linear layer.

In [7]:
import torch.nn as nn
import torch.nn.functional as f

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, text):
        # text = [sentence length, batch size]
        embedded = self.embedding(text)
        # embedded = [sentence length, batch size, embedding dim]
        embedded = embedded.permute(1, 0, 2)
        # embedded = [batch size, sentence length, embedding dim]
        pooled = f.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)
        # pooled = [batch size, embedding dim]
        return self.fc(pooled)

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


> Copy pre-trained vector to embedding layer

In [10]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.2463,  0.5749, -0.3656,  ...,  0.4434,  0.5794, -0.3972],
        [ 0.9874, -0.3359,  1.8736,  ..., -0.2993, -0.3686,  0.9624],
        [ 0.4455,  0.1342, -2.3334,  ...,  0.9246,  0.3400, -1.0022]])

> Khởi tạo giá trị 0 cho các từ chưa biết và các pad tokens

In [11]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

### Train the Model

> Optimizer

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

> Criterion

In [13]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [14]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [15]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [18]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 50s
	Train Loss: 0.686 | Train Acc: 61.27%
	 Val. Loss: 0.632 |  Val. Acc: 71.65%
Epoch: 02 | Epoch Time: 1m 48s
	Train Loss: 0.647 | Train Acc: 73.67%
	 Val. Loss: 0.513 |  Val. Acc: 75.55%
Epoch: 03 | Epoch Time: 1m 39s
	Train Loss: 0.573 | Train Acc: 79.74%
	 Val. Loss: 0.431 |  Val. Acc: 80.53%
Epoch: 04 | Epoch Time: 1m 37s
	Train Loss: 0.498 | Train Acc: 84.01%
	 Val. Loss: 0.391 |  Val. Acc: 83.64%
Epoch: 05 | Epoch Time: 1m 32s
	Train Loss: 0.432 | Train Acc: 86.85%
	 Val. Loss: 0.375 |  Val. Acc: 85.80%


In [19]:
model.load_state_dict(torch.load('./models/tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.385 | Test Acc: 85.35%


### User Input

In [20]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

> Negative review

In [21]:
predict_sentiment(model, "This film is terrible")

2.6169841405021543e-08

> Positive review

In [22]:
predict_sentiment(model, "This film is great")


1.0

### Next Step

> Tiếp theo, chúng ta sẽ sử dụng Convolution Neural Networks (CNNs) cho bài toán Sentiment Analysis.

## END